# 第六章 评估

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、 创建LLM应用](#二、-创建LLM应用)
     - [2.1 创建评估数据点](#2.1-创建评估数据点)
     - [2.2 创建测试用例数据](#2.2-创建测试用例数据)
     - [2.3 通过LLM生成测试用例](#2.3-通过LLM生成测试用例)
     - [2.4 组合用例数据](#2.4-组合用例数据)
 - [三、 人工评估](#三、-人工评估)
     - [3.1 如何评估新创建的实例](#3.1-如何评估新创建的实例)
     - [3.2 中文版](#3.2-中文版)
 - [四、 通过LLM进行评估实例](#四、-通过LLM进行评估实例)
     - [4.1  评估思路](#4.1--评估思路)
     - [4.2 结果分析](#4.2-结果分析)
     - [3.3 通过LLM进行评估实例](#3.3-通过LLM进行评估实例)


## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

## 二、 创建LLM应用
按照langchain链的方式进行构建

In [99]:
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储

#加载数据
file = './data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [5]:
#查看数据
import pandas as pd
test_data = pd.read_csv(file,header=None)
test_data

,0,1,2
0,NaN,name,description
1,0.0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
2,1.0,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
3,2.0,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
4,3.0,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
...,...,...,...
996,995.0,"Men's Classic Denim, Standard Fit",Crafted from premium denim that will last wash...
997,996.0,CozyPrint Sweater Fleece Pullover,The ultimate sweater fleece - made from superi...
998,997.0,Women's NRS Endurance Spray Paddling Pants,These comfortable and affordable splash paddli...
999,998.0,Women's Stop Flies Hoodie,This great-looking hoodie uses No Fly Zone Tec...


<span style="color:#00ff00">
这里我们调用LangChain中的高级封装<code>VectorstoreIndexCreator</code>类来创建向量存储。它的底层向量存储实现还是和之前一样采用<code>DocArrayInMemorySearch</code>类，不过可以提供一些高级功能。
</span>

In [100]:
from langchain_openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器
from langchain.vectorstores import DocArrayInMemorySearch #向量存储

embeddings = OpenAIEmbeddings(model='text-embedding-3-small') #嵌入模型

index = VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_cls=DocArrayInMemorySearch
).from_documents(data[:100]) # 考虑到时间及经济方面只向量化前100行数据

<span style="color:#00ff00">
这里我们通过参数<code>chain_type_kwargs = {"document_separator": "<<<<<>>>>>"}</code>设置了文档之间的分隔符，分隔符<code><<<<< >>>>></code>用于在prompt中清晰区分不同文档的边界，让LLM能够明确识别不同文档的内容。
</span>

In [101]:
from langchain.chains.retrieval_qa.base import RetrievalQA #检索QA链，在文档上进行检索
from langchain.chat_models import init_chat_model

#通过指定语言模型、链类型、检索器和我们要打印的详细程度来创建检索QA链
llm = init_chat_model(model="gpt-4o-mini",temperature = 0.0)

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### 2.1 创建评估数据点
我们需要做的第一件事是真正弄清楚我们<span style="color:red">想要评估它的一些数据点</span>，我们将介绍几种不同的方法来完成这个任务

1、将自己想出好的数据点作为例子，查看一些数据，然后想出例子问题和答案，以便以后用于评估

In [10]:
data[10]#查看这里的一些文档，我们可以对其中发生的事情有所了解

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [11]:
data[11]

Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

看起来第一个文档中有这个套头衫，第二个文档中有这个夹克，从这些细节中，我们可以创建一些例子查询和答案

### 2.2 创建测试用例数据


<span style="color:#00ff00">
手动创建测试用例。
</span>

In [102]:
# 同样，由于文档是英文，我们采用英文示例。

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

因此，我们可以问一个简单的问题，这个舒适的套头衫套装有侧口袋吗？，我们可以通过上面的内容看到，它确实有一些侧口袋，答案为是。

对于第二个文档，我们可以看到这件夹克来自某个系列，即down tech系列，答案是down tech系列。

### 2.3 通过LLM生成测试用例

<span style="color:#00ff00">
利用LangChain中的<code>QAGenerateChain</code>类，可以通过LLM自动创建测试用例。
</span>

In [103]:
from langchain.evaluation.qa import QAGenerateChain #导入QA生成链，它将接收文档，并从每个文档中创建一个问题答案对

example_gen_chain = QAGenerateChain.from_llm(init_chat_model(model="gpt-4o-mini",temperature = 0.0))#通过传递chat open AI语言模型来创建这个链

In [104]:
data[:5]

[Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries."),
 Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 1}, page_content=': 1\nname: Recycled 

<span style="color:#00ff00">
我们采用LongChain的新版api，使用<code>batch</code>方法处理多个文档，并最后将结果转换为所需格式。
</span>

In [108]:
inputs = [{"doc": doc} for doc in data[:5]]

# 使用batch方法处理多个文档
raw_results = example_gen_chain.batch(inputs)

In [109]:
raw_results

[{'doc': Document(metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries."),
  'qa_pairs': {'query': "What are the key features of the Women's Campside Oxfords, including materials, cons

<span style="color:#00ff00">
LLM生成的原始结果格式与我们自定义的格式不一致，因此需要进行转换。
</span>

In [110]:
# 将结果转换为所需格式
# new_examples现在格式为[{'query': '...', 'answer': '...'}]
new_examples = [{"query": r['qa_pairs']["query"], 
                 "answer": r['qa_pairs']["answer"]} for r in raw_results]

In [111]:
new_examples

[{'query': "What are the key features of the Women's Campside Oxfords, including materials, construction, and fit recommendations?",
  'answer': "The Women's Campside Oxfords are designed with a super-soft canvas material that provides a broken-in feel from the first wear. They feature thick cushioning, a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, and a vintage hunt, fish, and camping motif on the innersole. The shoes have a moderate arch contour and an EVA foam midsole for added cushioning and support. The outsole is made of molded rubber with a modified chain-tread pattern. For sizing, it is recommended to order the regular shoe size, and for half sizes not offered, to order up to the next whole size. The approximate weight of the shoes is 1 lb. 1 oz. per pair."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, and what materials are used in its construction?',
  'answer': 'The small size of the Recyc

<span style="color:#00ff00">
旧版api，使用的<code>apply_and_parse</code>方法将被废弃。
</span>

In [54]:
raw_results_2 = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
) #我们可以创建许多例子

/Users/xiaweichun/miniconda3/envs/llm/lib/python3.11/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [55]:
raw_results_2

[{'qa_pairs': {'query': "What are the key features and specifications of the Women's Campside Oxfords as described in the document?",
   'answer': "The Women's Campside Oxfords are ultracomfortable lace-to-toe shoes made from super-soft canvas, featuring thick cushioning and quality construction for a broken-in feel from the first wear. They should be ordered in regular shoe size, and for half sizes not offered, customers should order up to the next whole size. The shoes weigh approximately 1 lb. 1 oz. per pair and include a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, and a moderate arch contour. The EVA foam midsole provides cushioning and support, while the outsole is made of molded rubber with a modified chain-tread pattern."}},
 {'qa_pairs': {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, and what materials are used in its construction?',
   'a

In [57]:
new_examples_2 = [{"query": r['qa_pairs']["query"], 
                 "answer": r['qa_pairs']["answer"]} for r in raw_results_2]
new_examples_2

[{'query': "What are the key features and specifications of the Women's Campside Oxfords as described in the document?",
  'answer': "The Women's Campside Oxfords are ultracomfortable lace-to-toe shoes made from super-soft canvas, featuring thick cushioning and quality construction for a broken-in feel from the first wear. They should be ordered in regular shoe size, and for half sizes not offered, customers should order up to the next whole size. The shoes weigh approximately 1 lb. 1 oz. per pair and include a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, and a moderate arch contour. The EVA foam midsole provides cushioning and support, while the outsole is made of molded rubber with a modified chain-tread pattern."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, and what materials are used in its construction?',
  'answer': 'The small size of th

### 2.4 组合用例数据

In [64]:
examples += new_examples

In [65]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What are the key features and specifications of the Women's Campside Oxfords as described in the document?",
  'answer': "The Women's Campside Oxfords are ultracomfortable lace-to-toe shoes made from super-soft canvas, featuring thick cushioning and quality construction for a broken-in feel from the first wear. They should be ordered in regular shoe size, and for half sizes not offered, customers should order up to the next whole size. The shoes weigh approximately 1 lb. 1 oz. per pair and include a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, a vintage hunt, fish, and camping motif on the innersole, and a moderate arch contour. The EVA foam midsole provides cushioning and support, while the outsole is made of molded rubbe

接下来，我们可以利用先前的qa链对上述示例中的query进行查询，检验他们的输出是否符合预期

<span style="color:#00ff00">
调用<code>invoke</code>方法处理单个查询。
</span>

In [35]:
qa.invoke({"query": examples[0]["query"]})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.'}

<span style="color:#00ff00">
调用<code>batch</code>方法同时处理多个查询。
</span>

In [67]:
# 批处理多个查询
results = qa.batch([{"query": example["query"]} for example in examples])



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [68]:
results

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'result': 'Yes, the Cozy Comfort Pullover Set has side pockets in the pull-on pants.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "What are the key features and specifications of the Women's Campside Oxfords as described in the document?",
  'result': "The key features and specifications of the Women's Campside Oxfords are as follows:\n\n- **Description**: Ultralight lace-to-toe Oxford with a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first wear.\n- **Size & Fit**: Order regular shoe size. For half sizes not offered, order up to the next whole size.\n- **Approximate Weight**: 1 lb. 1 oz. per pair.\n- **Construction**:\n  - Soft canvas material for a broken-in feel and look.\n  - Comfortable EVA innersole with


### 2.5 中文版


#### 2.5.1 按照langchain链的方式进行构建

In [112]:
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储

#加载中文数据
file = './data/product_data.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [113]:
#查看数据
import pandas as pd
test_data = pd.read_csv(file,header=None)
test_data

,0,1
0,product_name,description
1,全自动咖啡机,规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' ...
2,电动牙刷,规格:\n一般大小 - 高度：9.5''，宽度：1''。\n\n为什么我们热爱它:\n我们的...
3,橙味维生素C泡腾片,规格:\n每盒含有20片。\n\n为什么我们热爱它:\n我们的橙味维生素C泡腾片是快速补充维...
4,无线蓝牙耳机,规格:\n单个耳机尺寸：1.5'' x 1.3''。\n\n为什么我们热爱它:\n这款无线蓝...
5,瑜伽垫,规格:\n尺寸：24'' x 68''。\n\n为什么我们热爱它:\n我们的瑜伽垫拥有出色的...
6,防水运动手表,规格:\n表盘直径：40mm。\n\n为什么我们热爱它:\n这款防水运动手表配备了心率监测和...
7,书籍:《机器学习基础》,规格:\n页数：580页。\n\n为什么我们热爱它:\n《机器学习基础》以易懂的语言讲解了机...
8,空气净化器,规格:\n尺寸：15'' x 15'' x 20''。\n\n为什么我们热爱它:\n我们的空...
9,陶瓷保温杯,规格:\n容量：350ml。\n\n为什么我们热爱它:\n我们的陶瓷保温杯设计优雅，保温效果...


In [114]:
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器
from langchain.vectorstores import DocArrayInMemorySearch #向量存储

index = VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [115]:
from langchain.chains.retrieval_qa.base import RetrievalQA #检索QA链，在文档上进行检索
from langchain.chat_models import init_chat_model

#通过指定语言模型、链类型、检索器和我们要打印的详细程度来创建检索QA链
llm = init_chat_model(model="gpt-4o-mini",temperature = 0.0)

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### 2.5.2 创建评估数据点


将自己想出好的数据点作为例子，查看一些数据，然后想出例子问题和答案，以便以后用于评估

In [73]:
data[10]#查看这里的一些文档，我们可以对其中发生的事情有所了解

Document(metadata={'source': './data/product_data.csv', 'row': 10}, page_content="product_name: 高清电视机\ndescription: 规格:\n尺寸：50''。\n\n为什么我们热爱它:\n我们的高清电视机拥有出色的画质和强大的音效，带来沉浸式的观看体验。\n\n材质与护理:\n使用干布清洁。\n\n构造:\n由塑料、金属和电子元件制成。\n\n其他特性:\n支持网络连接，可以在线观看视频。\n配备遥控器。\n在韩国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。")

In [74]:
data[11]

Document(metadata={'source': './data/product_data.csv', 'row': 11}, page_content="product_name: 旅行背包\ndescription: 规格:\n尺寸：18'' x 12'' x 6''。\n\n为什么我们热爱它:\n我们的旅行背包拥有多个实用的内外袋，轻松装下您的必需品，是短途旅行的理想选择。\n\n材质与护理:\n可以手洗，自然晾干。\n\n构造:\n由防水尼龙制成。\n\n其他特性:\n附带可调节背带和安全锁。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。")

看上面的第一个文档中有高清电视机，第二个文档中有旅行背包，从这些细节中，我们可以创建一些例子查询和答案

#### 2.5.3 创建测试用例数据


In [116]:
examples = [
    {
        "query": "高清电视机怎么进行护理？",
        "answer": "使用干布清洁。"
    },
    {
        "query": "旅行背包有内外袋吗？",
        "answer": "有。"
    }
]

#### 2.5.4 通过LLM生成测试用例

In [117]:
from langchain.evaluation.qa import QAGenerateChain #导入QA生成链，它将接收文档，并从每个文档中创建一个问题答案对

由于`QAGenerateChain`类中使用的`PROMPT`是英文，故我们继承`QAGenerateChain`类，将`PROMPT`加上“请使用中文输出”。

下面是`generate_chain.py`文件中的`QAGenerateChain`类的源码

In [118]:
"""LLM Chain specifically for generating examples for question answering."""
from __future__ import annotations

from typing import Any

from langchain.base_language import BaseLanguageModel
from langchain.chains.llm import LLMChain
from langchain.evaluation.qa.generate_prompt import PROMPT

class QAGenerateChain(LLMChain):
    """LLM Chain specifically for generating examples for question answering."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, **kwargs: Any) -> QAGenerateChain:
        """Load QA Generate Chain from LLM."""
        return cls(llm=llm, prompt=PROMPT, **kwargs)

In [119]:
PROMPT

PromptTemplate(input_variables=['doc'], input_types={}, partial_variables={}, template='You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\n{doc}\n<End Document>')

我们可以看到`PROMPT`为英文，下面我们将`PROMPT`添加上“请使用中文输出”

##### 2.5.4.1 <span style="color:#00ff00">新版api</span>

In [129]:
# 下面是langchain.evaluation.qa.generate_prompt中的源码，我们在template的最后加上“请使用中文输出”
# flake8: noqa
from langchain.output_parsers.regex import RegexParser
from langchain.prompts import PromptTemplate

template = """You are a teacher coming up with questions to ask on a quiz. 
Given the following document, please generate a question and answer based on that document.

Example Format:
<Begin Document>
...
<End Document>
QUESTION: question here
ANSWER: answer here

These questions should be detailed and be based explicitly on information in the document. Begin!

<Begin Document>
{doc}
<End Document>
请使用中文输出。
"""
PROMPT_new = PromptTemplate(
    input_variables=["doc"], template=template
)

PROMPT_new


PromptTemplate(input_variables=['doc'], input_types={}, partial_variables={}, template='You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\n{doc}\n<End Document>\n请使用中文输出。\n')

In [131]:
# 继承QAGenerateChain
class MyQAGenerateChain(QAGenerateChain):
    """LLM Chain specifically for generating examples for question answering."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, **kwargs: Any) -> QAGenerateChain:
        """Load QA Generate Chain from LLM."""
        return cls(llm=llm, prompt=PROMPT_new, **kwargs)

In [132]:
example_gen_chain = MyQAGenerateChain.from_llm(init_chat_model(model="gpt-4o-mini",temperature = 0.0))#通过传递chat open AI语言模型来创建这个链

In [141]:
data[:5]

[Document(metadata={'source': './data/product_data.csv', 'row': 0}, page_content="product_name: 全自动咖啡机\ndescription: 规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' x 15.2''。\n\n为什么我们热爱它:\n这款全自动咖啡机是爱好者的理想选择。 一键操作，即可研磨豆子并沏制出您喜爱的咖啡。它的耐用性和一致性使它成为家庭和办公室的理想选择。\n\n材质与护理:\n清洁时只需轻擦。\n\n构造:\n由高品质不锈钢制成。\n\n其他特性:\n内置研磨器和滤网。\n预设多种咖啡模式。\n在中国制造。\n\n有问题？ 请随时联系我们的客户服务团队，他们会解答您的所有问题。"),
 Document(metadata={'source': './data/product_data.csv', 'row': 1}, page_content="product_name: 电动牙刷\ndescription: 规格:\n一般大小 - 高度：9.5''，宽度：1''。\n\n为什么我们热爱它:\n我们的电动牙刷采用先进的刷头设计和强大的电机，为您提供超凡的清洁力和舒适的刷牙体验。\n\n材质与护理:\n不可水洗，只需用湿布清洁。\n\n构造:\n由食品级塑料和尼龙刷毛制成。\n\n其他特性:\n具有多种清洁模式和定时功能。\nUSB充电。\n在日本制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。"),
 Document(metadata={'source': './data/product_data.csv', 'row': 2}, page_content='product_name: 橙味维生素C泡腾片\ndescription: 规格:\n每盒含有20片。\n\n为什么我们热爱它:\n我们的橙味维生素C泡腾片是快速补充维生素C的理想方式。每片含有500mg的维生素C，可以帮助提升免疫力，保护您的健康。\n\n材质与护理:\n请存放在阴凉干燥的地方，避免阳光直射。\n\n构造:\n主要成分为维生素C和柠檬酸钠。\n\n其他特性:\n含有天然橙味。\n易于携带。\n在美国制造。\n\n有问

In [133]:
inputs = [{"doc": doc} for doc in data[:5]]

# 使用batch方法处理多个文档
raw_results = example_gen_chain.batch(inputs)

<span style="color:#00ff00">
此时的原始输出格式需要进行转换。我们采用<code>RegexParser</code>解析器进行转换。
</span>

In [134]:
raw_results 

[{'doc': Document(metadata={'source': './data/product_data.csv', 'row': 0}, page_content="product_name: 全自动咖啡机\ndescription: 规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' x 15.2''。\n\n为什么我们热爱它:\n这款全自动咖啡机是爱好者的理想选择。 一键操作，即可研磨豆子并沏制出您喜爱的咖啡。它的耐用性和一致性使它成为家庭和办公室的理想选择。\n\n材质与护理:\n清洁时只需轻擦。\n\n构造:\n由高品质不锈钢制成。\n\n其他特性:\n内置研磨器和滤网。\n预设多种咖啡模式。\n在中国制造。\n\n有问题？ 请随时联系我们的客户服务团队，他们会解答您的所有问题。"),
  'text': "QUESTION: 这款全自动咖啡机的尺寸有哪些？请分别列出大型和中型的尺寸。\nANSWER: 大型的尺寸是13.8'' x 17.3''，中型的尺寸是11.5'' x 15.2''。"},
 {'doc': Document(metadata={'source': './data/product_data.csv', 'row': 1}, page_content="product_name: 电动牙刷\ndescription: 规格:\n一般大小 - 高度：9.5''，宽度：1''。\n\n为什么我们热爱它:\n我们的电动牙刷采用先进的刷头设计和强大的电机，为您提供超凡的清洁力和舒适的刷牙体验。\n\n材质与护理:\n不可水洗，只需用湿布清洁。\n\n构造:\n由食品级塑料和尼龙刷毛制成。\n\n其他特性:\n具有多种清洁模式和定时功能。\nUSB充电。\n在日本制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。"),
  'text': 'QUESTION: 这款电动牙刷的高度和宽度分别是多少？它的主要材质是什么？\nANSWER: 这款电动牙刷的高度为9.5英寸，宽度为1英寸。主要材质为食品级塑料和尼龙刷毛。'},
 {'doc': Document(metadata={'source': './data/product_data.csv', 'row': 2},

<span style="color:#00ff00">
<code>RegexParser</code> 是一个用于从文本中提取结构化信息的解析器，它按照正则表达式模式：<code>r"QUESTION: (.*?)\nANSWER: (.*)"</code>匹配以 "QUESTION: "开头的文本，捕获问题内容 (.*?)，直到遇到换行符，然后匹配 "ANSWER: " 标记，捕获答案内容 (.*)（剩余所有文本）。
</span>

In [135]:
output_parser = RegexParser(
    regex=r"QUESTION: (.*?)\nANSWER: (.*)", output_keys=["query", "answer"]
)

In [145]:
raw_results_text = [r['text'] for r in raw_results]

new_examples = output_parser.batch(raw_results_text)

In [146]:
new_examples

[{'query': '这款全自动咖啡机的尺寸有哪些？请分别列出大型和中型的尺寸。',
  'answer': "大型的尺寸是13.8'' x 17.3''，中型的尺寸是11.5'' x 15.2''。"},
 {'query': '这款电动牙刷的高度和宽度分别是多少？它的主要材质是什么？',
  'answer': '这款电动牙刷的高度为9.5英寸，宽度为1英寸。主要材质为食品级塑料和尼龙刷毛。'},
 {'query': '橙味维生素C泡腾片的主要成分是什么？每片含有多少毫克的维生素C？  ',
  'answer': '橙味维生素C泡腾片的主要成分为维生素C和柠檬酸钠，每片含有500mg的维生素C。'},
 {'query': '这款无线蓝牙耳机的电池续航力是多少小时？并且它具备哪些特别的功能？',
  'answer': '这款无线蓝牙耳机的电池续航力为长达8小时，具备降噪技术、快速充电功能和内置麦克风，支持接听电话。'},
 {'query': '这款瑜伽垫的尺寸是多少？它的材质是什么？  ',
  'answer': '这款瑜伽垫的尺寸是24英寸 x 68英寸，材质是环保PVC材料。'}]

##### 2.5.4.2 <span style="color:#00ff00">旧版api</span>

In [138]:
# 下面是langchain.evaluation.qa.generate_prompt中的源码，我们在template的最后加上“请使用中文输出”
# flake8: noqa
from langchain.output_parsers.regex import RegexParser
from langchain.prompts import PromptTemplate

template = """You are a teacher coming up with questions to ask on a quiz. 
Given the following document, please generate a question and answer based on that document.

Example Format:
<Begin Document>
...
<End Document>
QUESTION: question here
ANSWER: answer here

These questions should be detailed and be based explicitly on information in the document. Begin!

<Begin Document>
{doc}
<End Document>
请使用中文输出。
"""
output_parser = RegexParser(
    regex=r"QUESTION: (.*?)\nANSWER: (.*)", output_keys=["query", "answer"]
)

# 直接在提示模版中应用我们的解析器
PROMPT_ch = PromptTemplate(
    input_variables=["doc"], template=template, output_parser=output_parser
)

PROMPT_ch


PromptTemplate(input_variables=['doc'], input_types={}, output_parser=RegexParser(regex='QUESTION: (.*?)\\nANSWER: (.*)', output_keys=['query', 'answer']), partial_variables={}, template='You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\n{doc}\n<End Document>\n请使用中文输出。\n')

In [139]:
# 继承QAGenerateChain
class MyQAGenerateChain(QAGenerateChain):
    """LLM Chain specifically for generating examples for question answering."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, **kwargs: Any) -> QAGenerateChain:
        """Load QA Generate Chain from LLM."""
        return cls(llm=llm, prompt=PROMPT_ch, **kwargs)

In [140]:
example_gen_chain = MyQAGenerateChain.from_llm(init_chat_model(model="gpt-4o-mini",temperature = 0.0))#通过传递chat open AI语言模型来创建这个链

<span style="color:#00ff00">
旧版api可以直接使用<code>apply_and_parse</code>方法来生成符合预期格式的用例数据。
</span>

In [143]:
new_examples_2 = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
) 

/Users/xiaweichun/miniconda3/envs/llm/lib/python3.11/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [147]:
new_examples_2 #查看用例数据

[{'query': '这款全自动咖啡机的尺寸有哪些？并且它适合在哪些场合使用？',
  'answer': "这款全自动咖啡机有两种尺寸：大型尺寸为13.8'' x 17.3''，中型尺寸为11.5'' x 15.2''。它是爱好者的理想选择，适合家庭和办公室使用。"},
 {'query': '这款电动牙刷的高度和宽度分别是多少？它的主要材质是什么？',
  'answer': '这款电动牙刷的高度为9.5英寸，宽度为1英寸。主要材质为食品级塑料和尼龙刷毛。'},
 {'query': '橙味维生素C泡腾片的主要成分是什么？每片含有多少毫克的维生素C？  ',
  'answer': '橙味维生素C泡腾片的主要成分为维生素C和柠檬酸钠，每片含有500mg的维生素C。'},
 {'query': '这款无线蓝牙耳机的电池续航力是多少小时？并且它具备哪些特别的功能？',
  'answer': '这款无线蓝牙耳机的电池续航力为长达8小时，具备降噪技术、快速充电功能和内置麦克风，支持接听电话。'},
 {'query': '瑜伽垫的尺寸是多少？它的材质是什么？  ', 'answer': "瑜伽垫的尺寸是24'' x 68''，材质是环保PVC材料。"}]

#### 2.5.4.3 <span style="color:#00ff00">总结</span>


<span style="color:#00ff00">
相比较于新版api，旧版api的生成用例数据的方式更加简单，只需要在提示模版中应用我们的解析器即可，不需要额外的对输出进行一次解析器处理。
</span>


#### 2.6 组合用例数据

In [150]:
examples += new_examples

In [156]:
examples

[{'query': '高清电视机怎么进行护理？', 'answer': '使用干布清洁。'},
 {'query': '旅行背包有内外袋吗？', 'answer': '有。'},
 {'query': '这款全自动咖啡机的尺寸有哪些？请分别列出大型和中型的尺寸。',
  'answer': "大型的尺寸是13.8'' x 17.3''，中型的尺寸是11.5'' x 15.2''。"},
 {'query': '这款电动牙刷的高度和宽度分别是多少？它的主要材质是什么？',
  'answer': '这款电动牙刷的高度为9.5英寸，宽度为1英寸。主要材质为食品级塑料和尼龙刷毛。'},
 {'query': '橙味维生素C泡腾片的主要成分是什么？每片含有多少毫克的维生素C？  ',
  'answer': '橙味维生素C泡腾片的主要成分为维生素C和柠檬酸钠，每片含有500mg的维生素C。'},
 {'query': '这款无线蓝牙耳机的电池续航力是多少小时？并且它具备哪些特别的功能？',
  'answer': '这款无线蓝牙耳机的电池续航力为长达8小时，具备降噪技术、快速充电功能和内置麦克风，支持接听电话。'},
 {'query': '这款瑜伽垫的尺寸是多少？它的材质是什么？  ',
  'answer': '这款瑜伽垫的尺寸是24英寸 x 68英寸，材质是环保PVC材料。'}]

<span style="color:#00ff00">
同样可以使用<code>invoke</code>和<code>batch</code>方法来查询用例数据。
</span>

In [151]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': '高清电视机怎么进行护理？', 'result': '高清电视机的护理方法是使用干布清洁。'}

In [159]:
# 使用batch方法处理多个文档

qa.batch(([{"query": example["query"]} for example in examples]))



> Entering new RetrievalQA chain...

> Entering new RetrievalQA chain...



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


[{'query': '高清电视机怎么进行护理？', 'result': '高清电视机的护理方法是使用干布清洁。'},
 {'query': '旅行背包有内外袋吗？', 'result': '是的，旅行背包拥有多个实用的内外袋。'},
 {'query': '这款全自动咖啡机的尺寸有哪些？请分别列出大型和中型的尺寸。',
  'result': "这款全自动咖啡机的尺寸如下：\n- 大型：13.8'' x 17.3''。\n- 中型：11.5'' x 15.2''。"},
 {'query': '这款电动牙刷的高度和宽度分别是多少？它的主要材质是什么？',
  'result': '这款电动牙刷的高度是9.5英寸，宽度是1英寸。它的主要材质是食品级塑料和尼龙刷毛。'},
 {'query': '橙味维生素C泡腾片的主要成分是什么？每片含有多少毫克的维生素C？  ',
  'result': '橙味维生素C泡腾片的主要成分为维生素C和柠檬酸钠。每片含有500mg的维生素C。'},
 {'query': '这款无线蓝牙耳机的电池续航力是多少小时？并且它具备哪些特别的功能？',
  'result': '这款无线蓝牙耳机的电池续航力是长达8小时。它具备的特别功能包括降噪技术、快速充电功能和内置麦克风，支持接听电话。'},
 {'query': '这款瑜伽垫的尺寸是多少？它的材质是什么？  ',
  'result': "瑜伽垫的尺寸是24'' x 68''，材质是环保PVC材料。"}]

## 三、 人工评估
现在有了这些示例，但是我们如何<span style="color:red">评估正在发生的事情</span>呢？

我们运行一个示例通过链，并查看它产生的输出。在这里我们传递一个查询，然后我们得到一个答案。实际上正在发生的事情，进入语言模型的实际提示是什么？它检索的文档是什么？   中间结果是什么？仅仅查看最终答案通常不足以了解链中出现了什么问题或可能出现了什么问题

<span style="color:#00ff00">
LingChain的<code>debug</code>工具可以了解运行一个实例通过链中间所经历的步骤
</span>

In [160]:
import langchain
langchain.debug = True

In [161]:
qa.invoke(examples[1]["query"])#重新运行与上面相同的示例，可以看到它开始打印出更多的信息

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "旅行背包有内外袋吗？"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "旅行背包有内外袋吗？",
  "context": "product_name: 旅行背包\ndescription: 规格:\n尺寸：18'' x 12'' x 6''。\n\n为什么我们热爱它:\n我们的旅行背包拥有多个实用的内外袋，轻松装下您的必需品，是短途旅行的理想选择。\n\n材质与护理:\n可以手洗，自然晾干。\n\n构造:\n由防水尼龙制成。\n\n其他特性:\n附带可调节背带和安全锁。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。<<<<>>>>>product_name: 迷你书架\ndescription: 规格:\n尺寸：20'' x 8'' x 24''。\n\n为什么我们热爱它:\n我们的迷你书架占地面积小，设计简约，能够有效存放和展示您的书籍。\n\n材质与护理:\n使用干布擦拭。\n\n构造:\n由实木制成。\n\n其他特性:\n拥有3层收纳空间。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。<<<<>>>>>product_name: 摄像头保护套\ndescription: 规格:\n适用于各种品牌和型号的摄像头。\n\n为什么我们热爱它:\n我们的摄像头保护套可以有效保护您的摄像头不受灰尘和刮擦的影响。\n\n材质与护理:\n使用湿布擦拭。\n\n构造:\n由耐磨的合成纤维制成。\n\n其他特性:\n设计简洁，易于安装。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。<<<<>>>>

{'query': '旅行背包有内外袋吗？', 'result': '是的，旅行背包拥有多个实用的内外袋。'}

我们可以看到它首先深入到检索QA链中，然后它进入了一些文档链（<span style="color:#00ff00">可以看到，对于不同的查询，进入的文档链是不同的，并且符合我们的预期</span>）。如上所述，我们正在使用stuff方法，现在我们正在传递这个上下文，可以看到，这个上下文是由我们<span style="color:red">检索到的不同文档创建的</span>。因此，在进行问答时，当返回错误结果时，通常不是语言模型本身出错了，实际上是**检索步骤**出错了，仔细查看问题的确切内容和上下文可以帮助调试出错的原因。    

然后，我们可以再向下一级，看看进入语言模型的确切内容，以及 OpenAI 自身，在这里，我们可以看到传递的完整提示，我们有一个系统消息，有所使用的提示的描述，这是问题回答链使用的提示，我们可以看到提示打印出来，使用以下上下文片段回答用户的问题。
如果您不知道答案，只需说您不知道即可，不要试图编造答案。然后我们看到一堆之前插入的上下文，我们还可以看到有关实际返回类型的更多信息。我们不仅仅返回一个答案，还有token的使用情况，可以了解到token数的使用情况


由于这是一个相对简单的链，我们现在可以看到最终的响应，舒适的毛衣套装，条纹款，有侧袋，正在起泡，通过链返回给用户，我们刚刚讲解了如何查看和调试单个输入到该链的情况。




## 四、 通过LLM进行评估实例

### 4.1 处理所有新创建的实例


首先我们运行链条来处理所有示例，然后查看输出。

In [162]:
# 我们需要为所有示例创建预测，关闭调试模式，以便不将所有内容打印到屏幕上
langchain.debug = False

In [174]:
examples

[{'query': '高清电视机怎么进行护理？', 'answer': '使用干布清洁。'},
 {'query': '旅行背包有内外袋吗？', 'answer': '有。'},
 {'query': '这款全自动咖啡机的尺寸有哪些？请分别列出大型和中型的尺寸。',
  'answer': "大型的尺寸是13.8'' x 17.3''，中型的尺寸是11.5'' x 15.2''。"},
 {'query': '这款电动牙刷的高度和宽度分别是多少？它的主要材质是什么？',
  'answer': '这款电动牙刷的高度为9.5英寸，宽度为1英寸。主要材质为食品级塑料和尼龙刷毛。'},
 {'query': '橙味维生素C泡腾片的主要成分是什么？每片含有多少毫克的维生素C？  ',
  'answer': '橙味维生素C泡腾片的主要成分为维生素C和柠檬酸钠，每片含有500mg的维生素C。'},
 {'query': '这款无线蓝牙耳机的电池续航力是多少小时？并且它具备哪些特别的功能？',
  'answer': '这款无线蓝牙耳机的电池续航力为长达8小时，具备降噪技术、快速充电功能和内置麦克风，支持接听电话。'},
 {'query': '这款瑜伽垫的尺寸是多少？它的材质是什么？  ',
  'answer': '这款瑜伽垫的尺寸是24英寸 x 68英寸，材质是环保PVC材料。'}]

In [175]:
predictions = qa.batch(([{"query": example["query"]} for example in examples]))



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [176]:
predictions

[{'query': '高清电视机怎么进行护理？', 'result': '高清电视机的护理方法是使用干布清洁。'},
 {'query': '旅行背包有内外袋吗？', 'result': '是的，旅行背包拥有多个实用的内外袋。'},
 {'query': '这款全自动咖啡机的尺寸有哪些？请分别列出大型和中型的尺寸。',
  'result': "这款全自动咖啡机的尺寸如下：\n- 大型：13.8'' x 17.3''。\n- 中型：11.5'' x 15.2''。"},
 {'query': '这款电动牙刷的高度和宽度分别是多少？它的主要材质是什么？',
  'result': '这款电动牙刷的高度是9.5英寸，宽度是1英寸。它的主要材质是食品级塑料和尼龙刷毛。'},
 {'query': '橙味维生素C泡腾片的主要成分是什么？每片含有多少毫克的维生素C？  ',
  'result': '橙味维生素C泡腾片的主要成分为维生素C和柠檬酸钠。每片含有500mg的维生素C。'},
 {'query': '这款无线蓝牙耳机的电池续航力是多少小时？并且它具备哪些特别的功能？',
  'result': '这款无线蓝牙耳机的电池续航力是长达8小时。它具备的特别功能包括降噪技术、快速充电功能和内置麦克风，支持接听电话。'},
 {'query': '这款瑜伽垫的尺寸是多少？它的材质是什么？  ',
  'result': "瑜伽垫的尺寸是24'' x 68''，材质是环保PVC材料。"}]

### 4.2 利用QA评估链进行评估

<span style="color:#00ff00">
我们利用LangChain库的<code>QAEvalChain</code>类，构建一条评估链对qa链的输出进行自动评估。评估链将评估qa链生成的预测答案对比真实答案是否正确。
</span>

In [181]:
from langchain.evaluation.qa import QAEvalChain #导入QA问题回答，评估链

#通过调用chatGPT进行评估
llm = init_chat_model(model="gpt-4o",temperature = 0.0)
eval_chain = QAEvalChain.from_llm(llm)

In [183]:
#在此链上调用evaluate，进行评估
graded_outputs = eval_chain.evaluate(examples, predictions)

In [184]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

### 4.3  评估思路


我们将循环遍历所有示例，并打印出示例中的`原始问题`、`真实答案`、qa链的`预测答案`以及评估链的`评估结果`。这可以详细了解每个示例。

In [186]:
#我们将传入示例和预测，得到一堆分级输出，循环遍历它们打印答案
for i, eg in enumerate(examples):
    print(f"例 {i}:")
    print("问题: " + examples[i]['query'])
    print("真实答案: " + examples[i]['answer'])
    print("预测答案: " + predictions[i]['result'])
    print("预测成绩: " + graded_outputs[i]['results'])
    print()

例 0:
问题: 高清电视机怎么进行护理？
真实答案: 使用干布清洁。
预测答案: 高清电视机的护理方法是使用干布清洁。
预测成绩: CORRECT

例 1:
问题: 旅行背包有内外袋吗？
真实答案: 有。
预测答案: 是的，旅行背包拥有多个实用的内外袋。
预测成绩: CORRECT

例 2:
问题: 这款全自动咖啡机的尺寸有哪些？请分别列出大型和中型的尺寸。
真实答案: 大型的尺寸是13.8'' x 17.3''，中型的尺寸是11.5'' x 15.2''。
预测答案: 这款全自动咖啡机的尺寸如下：
- 大型：13.8'' x 17.3''。
- 中型：11.5'' x 15.2''。
预测成绩: CORRECT

例 3:
问题: 这款电动牙刷的高度和宽度分别是多少？它的主要材质是什么？
真实答案: 这款电动牙刷的高度为9.5英寸，宽度为1英寸。主要材质为食品级塑料和尼龙刷毛。
预测答案: 这款电动牙刷的高度是9.5英寸，宽度是1英寸。它的主要材质是食品级塑料和尼龙刷毛。
预测成绩: CORRECT

例 4:
问题: 橙味维生素C泡腾片的主要成分是什么？每片含有多少毫克的维生素C？  
真实答案: 橙味维生素C泡腾片的主要成分为维生素C和柠檬酸钠，每片含有500mg的维生素C。
预测答案: 橙味维生素C泡腾片的主要成分为维生素C和柠檬酸钠。每片含有500mg的维生素C。
预测成绩: CORRECT

例 5:
问题: 这款无线蓝牙耳机的电池续航力是多少小时？并且它具备哪些特别的功能？
真实答案: 这款无线蓝牙耳机的电池续航力为长达8小时，具备降噪技术、快速充电功能和内置麦克风，支持接听电话。
预测答案: 这款无线蓝牙耳机的电池续航力是长达8小时。它具备的特别功能包括降噪技术、快速充电功能和内置麦克风，支持接听电话。
预测成绩: CORRECT

例 6:
问题: 这款瑜伽垫的尺寸是多少？它的材质是什么？  
真实答案: 这款瑜伽垫的尺寸是24英寸 x 68英寸，材质是环保PVC材料。
预测答案: 瑜伽垫的尺寸是24'' x 68''，材质是环保PVC材料。
预测成绩: CORRECT



### 4.4 结果分析
对于每个示例，它看起来都是正确的，让我们看看第一个例子。

这里的问题是，高清电视机怎么进行护理？真正的答案是使用干布清洁。模型预测的答案也是使用干布清洁。因此，我们可以确认这是一个正确的答案。评估链也将其评为正确。 

### 4.5 使用模型评估的优势

模型的回答可以是任意字符串，并不存在唯一最优的标准答案。只要不同的变体表达了相同的语义，就应被评为相似。使用正则表达式进行精确匹配会导致语义信息的丧失，许多现有的评估指标尚不完善。目前，<span style="color:red">利用语言模型进行评估</span>是最有趣且最受欢迎的方法之一。